In [1]:
import pandas as pd
from pathlib import Path

In [2]:
results = pd.read_csv('results.csv')
results.head()

,meta,hid_num,epochs,worker,time_started,time_completed,duration_seconds,path
0,0.00,512,20,1,2021-11-21 08:14:42,2021-11-21 09:12:59,3497.0,./results/0.0_512_20
1,0.50,2048,20,1,2021-11-21 09:12:59,2021-11-21 10:11:25,3506.0,./results/0.5_2048_20
2,1.35,512,20,1,2021-11-21 10:11:25,2021-11-21 11:09:59,3514.0,./results/1.35_512_20
3,0.00,2048,40,1,2021-11-21 11:09:59,2021-11-21 13:08:14,7095.0,./results/0.0_2048_40
4,1.00,512,40,1,2021-11-21 13:08:14,2021-11-21 15:04:33,6979.0,./results/1.0_512_40


In [3]:
for row in results.iterrows():
    ix, metadata = row
    
    # path of the csv with the results for each experiment
    # there's only one csv per experiment (i.e., per folder)
    path_csv = Path.glob(Path(metadata['path']), '*.csv').__next__()
    
    # get last row of the iterations of each experiment
    last_iteration = pd.read_csv(path_csv).iloc[-1]
    
    # get accuracy values
    acc_series = last_iteration.filter(regex='acc_test_tsk')
    
    # get loss values
    loss_series = last_iteration.filter(regex='loss_test_tsk')
    
    # statistics
    statistics = {"loss_std": loss_series.std(), "acc_std" : acc_series.std(), "loss_mean": loss_series.mean(),"acc_mean" : acc_series.mean()}
    
    statistics_series = pd.DataFrame(statistics, index=[ix])
    try:
        df_stats = pd.concat([df_stats, statistics_series])
    except NameError:
        df_stats = pd.DataFrame(statistics_series, index=[ix])

results = results.join(df_stats)

    
    

In [9]:
results.sort_values(by=['loss_std'], ascending=True)

,meta,hid_num,epochs,worker,time_started,time_completed,duration_seconds,path,loss_std,acc_std,loss_mean,acc_mean
23,1.35,1024,20,2,2021-11-21 10:11:50,2021-11-21 11:09:43,3473.0,./results/1.35_1024_20,0.042085,1.639069,0.272361,92.013333
93,3.00,2048,10,5,2021-11-21 19:35:11,2021-11-21 20:05:32,1821.0,./results/3.0_2048_10,0.055371,1.909544,0.219492,93.540000
112,2.00,1024,10,6,2021-11-21 19:08:31,2021-11-21 19:38:01,1770.0,./results/2.0_1024_10,0.069381,2.389569,0.283409,91.300000
114,1.50,1024,20,6,2021-11-21 20:07:40,2021-11-21 21:07:00,3560.0,./results/1.5_1024_20,0.082316,2.924506,0.255208,92.578333
25,1.00,1024,40,2,2021-11-21 13:09:24,2021-11-21 15:05:57,6993.0,./results/1.0_1024_40,0.088862,3.203983,0.258478,92.556667
...,...,...,...,...,...,...,...,...,...,...,...,...
68,2.00,4096,40,4,2021-11-21 16:02:53,2021-11-21 17:59:12,6979.0,./results/2.0_4096_40,11.874331,42.414396,5.797261,69.586667
24,0.00,4096,40,2,2021-11-21 11:09:43,2021-11-21 13:09:24,7181.0,./results/0.0_4096_40,13.647942,35.709745,9.842900,24.970000
26,1.35,4096,40,2,2021-11-21 15:05:57,2021-11-21 17:04:55,7138.0,./results/1.35_4096_40,14.244657,39.222391,7.201571,70.955000
36,1.50,4096,40,2,2021-11-22 01:26:49,2021-11-22 03:23:50,7021.0,./results/1.5_4096_40,14.645050,41.409487,7.179057,69.808333


In [10]:
results.to_csv('results_with_stats.csv', index=False)

,loss_std,acc_std,loss_mean,acc_mean
0,6.147175,35.546202,2.654411,82.115
